# ⛓ Chain 

In [ ]:
#| default_exp chains

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import asyncio, web3, os
from dataclasses import dataclass
from functools import wraps, reduce, partial
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from typing import List, TypeVar, Callable, Optional, Tuple, Dict, Union
from fastcore.utils import patch
from web3 import Web3, HTTPProvider, AsyncWeb3, AsyncHTTPProvider, Account
from web3.eth.async_eth import AsyncContract
from web3.eth import Contract
from web3.manager import RequestManager, RequestBatcher
from sugar.config import ChainSettings, make_op_chain_settings, make_base_chain_settings
from sugar.helpers import normalize_address, MAX_UINT256, float_to_uint256, apply_slippage, get_future_timestamp
from sugar.abi import sugar, slipstream, price_oracle, router, quoter
from sugar.token import Token
from sugar.pool import LiquidityPool, LiquidityPoolForSwap
from sugar.price import Price
from sugar.deposit import Deposit
from sugar.helpers import ADDRESS_ZERO, chunk, normalize_address, Pair, find_all_paths
from sugar.quote import Quote, RouteInput, prepare_route


In [ ]:
# monkey patching how web3 handles errors in batched requests
# re: https://github.com/ethereum/web3.py/issues/3657
original_format_batched_response = RequestManager._format_batched_response
def safe_format_batched_response(*args):
    try: return original_format_batched_response(*args)
    except Exception as e: return e
RequestManager._format_batched_response = safe_format_batched_response

## Chain implementation 

In [ ]:
#| export

T = TypeVar('T')

def require_context(f: Callable[..., T]) -> Callable[..., T]:
    @wraps(f)
    def wrapper(self: 'CommonChain', *args, **kwargs) -> T:
        if not self._in_context: raise RuntimeError("Chain methods can only be accessed within 'async with' block")
        return f(self, *args, **kwargs)
    return wrapper

def require_async_context(f: Callable[..., T]) -> Callable[..., T]:
    @wraps(f)
    async def wrapper(self: 'CommonChain', *args, **kwargs) -> T:
        if not self._in_context: raise RuntimeError("Chain methods can only be accessed within 'async with' block")
        return await f(self, *args, **kwargs)
    return wrapper


class CommonChain:
    @property
    def account(self) -> Account: return self.web3.eth.account.from_key(os.getenv("SUGAR_PK"))

    def __init__(self, settings: ChainSettings):
        self.settings, self._in_context = settings, False
    
    def prepare_tokens(self, tokens: List[Tuple], listed_only: bool) -> List[Token]:
        native = Token.make_native_token(self.settings.native_token_symbol, self.settings.wrapped_native_token_addr, self.settings.native_token_decimals)
        ts = list(map(lambda t: Token.from_tuple(t), tokens))
        return [native] + (list(filter(lambda t: t.listed, ts)) if listed_only else ts)
    
    def _prepare_prices(self, tokens: List[Token], rates: List[int]) -> Dict[str, int]:
        # token_address => normalized rate
        result = {}
        # rates are returned multiplied by eth decimals + the difference in decimals to eth
        # we want them all normalized to 18 decimals
        for cnt, rate in enumerate(rates):
            t, eth_decimals = tokens[cnt], self.settings.native_token_decimals
            if t.decimals == eth_decimals: nr = rate
            elif t.decimals < eth_decimals: nr = rate // (10 ** (eth_decimals - t.decimals))
            else: nr = rate * (10 ** (t.decimals - eth_decimals))
            result[t.token_address] = nr
        return result

    def prepare_prices(self, tokens: List[Token], prices: List[int]) -> List[Price]:
        """Get prices for tokens in target stable token"""
        eth_decimals = self.settings.native_token_decimals
        # all rates in EHT: token => rate
        rates_in_eth = self._prepare_prices(tokens, prices)
        eth_rate, usd_rate = rates_in_eth[self.settings.native_token_symbol], rates_in_eth[self.settings.stable_token_addr]
        # this gives us the price of 1 eth in usd with 18 decimals precision
        eth_usd_price = (eth_rate * 10 ** eth_decimals) // usd_rate
        # finally convert to prices in terms of stable
        return [Price(token=t, price=(rates_in_eth[t.token_address] * eth_usd_price // 10 ** eth_decimals) / 10 ** eth_decimals) for t in tokens]
    
    def prepare_pools(self, pools: List[Tuple], tokens: List[Token], prices: List[Price]) -> List[LiquidityPool]:
        tokens, prices = {t.token_address: t for t in tokens}, {price.token.token_address: price for price in prices}
        return list(filter(lambda p: p is not None, map(lambda p: LiquidityPool.from_tuple(p, tokens, prices), pools)))
    
    def prepare_pools_for_swap(self, pools: List[Tuple], tokens: List[Token]) -> List[LiquidityPoolForSwap]:
        tokens = {t.token_address: t for t in tokens}
        return list(filter(lambda p: p is not None, map(lambda p: LiquidityPoolForSwap.from_tuple(p, tokens), pools)))
    
    def prepare_quote_batch(self, batcher: RequestBatcher, pools: List[List[LiquidityPoolForSwap]], amount_in: int, paths: List[List[Tuple]]):
        for i, path in enumerate(paths):
            #print(f">>>>>>>>>> processing pools: {pools}")
            rt = prepare_route([RouteInput(pool=p, reversed=p.token0.token_address != path[i][0]) for i, p in enumerate(pools[i])])
            #print(f">>>>>>>>>> calling quoteExactInput with {amount_in} {rt.encoded}")
            batcher.add(self.quoter.functions.quoteExactInput(rt.encoded, amount_in))
        return batcher

    def prepare_quotes(self, from_token:Token, to_token:Token, amount_in: int, path_pools: List[List[LiquidityPoolForSwap]], responses):
        quotes = []
        for i, r in enumerate(responses):
            if isinstance(r, Exception): continue
            else: quotes.append(Quote(from_token=from_token, to_token=to_token, path=path_pools[i], amount_in=amount_in, amount_out=r[0]))
        return quotes
    
    def get_paths_for_quote(self, from_token: Token, to_token: Token, pools: List[LiquidityPoolForSwap], exclude_tokens: List[str]) -> List[List[Tuple]]:
        exclude_tokens_set = set(map(lambda t: normalize_address(t), exclude_tokens))

        if from_token.token_address in exclude_tokens: exclude_tokens_set.remove(from_token.token_address)
        if to_token.token_address in exclude_tokens: exclude_tokens_set.remove(to_token.token_address)

        pairs = [Pair(p.token0.token_address, p.token1.token_address, p.lp) for p in pools]
        paths = find_all_paths(pairs, from_token.wrapped_token_address or from_token.token_address, to_token.wrapped_token_address or to_token.token_address)
        # filter out paths with excluded tokens
        return list(filter(lambda p: len(set(map(lambda t: t[0], p)) & exclude_tokens_set) == 0, paths))

### Aync chain

In [ ]:
#| export

class AsyncChain(CommonChain):
    web3: AsyncWeb3
    sugar: AsyncContract
    slipstream: AsyncContract
    router: AsyncContract
    prices: AsyncContract
    quoter: AsyncContract


    async def __aenter__(self):
        """Async context manager entry"""
        self._in_context = True
        self.web3 = AsyncWeb3(AsyncHTTPProvider(self.settings.rpc_uri))
        self.sugar = self.web3.eth.contract(address=self.settings.sugar_contract_addr, abi=sugar)
        self.slipstream = self.web3.eth.contract(address=self.settings.slipstream_contract_addr, abi=slipstream)
        self.prices = self.web3.eth.contract(address=self.settings.price_oracle_contract_addr, abi=price_oracle)
        self.router = self.web3.eth.contract(address=self.settings.router_contract_addr, abi=router)
        self.quoter = self.web3.eth.contract(address=self.settings.quoter_contract_addr, abi=quoter)
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """Async context manager exit"""
        self._in_context = False
        await self.web3.provider.disconnect()
        return None

    @require_async_context
    async def get_all_tokens(self, listed_only: bool = True) -> List[Token]:
        # TODO: pagination for tokens
        tokens = await self.sugar.functions.tokens(1000, 0, ADDRESS_ZERO, []).call()
        return self.prepare_tokens(tokens, listed_only)
    
    # @cache_in_seconds(ORACLE_PRICES_CACHE_MINUTES * 60)
    async def _get_prices(self, tokens: Tuple[Token]) -> List[int]:
        # token_address => normalized rate
        return await self.prices.functions.getManyRatesToEthWithCustomConnectors(
            list(map(lambda t: t.wrapped_token_address or t.token_address, tokens)),
            False, # use wrappers
            self.settings.connector_tokens_addrs,
            10 # threshold_filter
        ).call()

    @require_async_context
    async def get_prices(self, tokens: List[Token]) -> List[Price]:
        """Get prices for tokens in target stable token"""

        # batches starts as a list of lists 
        batches = await asyncio.gather(
            *map(
                # XX: lists are not cacheable, convert them to tuples so lru cache is happy
                lambda ts: self._get_prices(tuple(ts)),
                list(chunk(tokens, self.settings.price_batch_size)),
            )
        )
        return self.prepare_prices(tokens, sum(batches, []))

    @require_async_context
    async def get_pools(self, for_swaps: bool = False) -> List[LiquidityPool]:
        pools, offset, limit = [], 0, self.settings.pool_page_size
        tokens = await self.get_all_tokens()
        
        while True:
            f = self.sugar.functions.all if not for_swaps else self.sugar.functions.forSwaps
            pools_batch = await f(limit, offset).call()
            pools += pools_batch
            if len(pools_batch) < limit: break
            else: offset += limit

        return self.prepare_pools(pools, tokens, await self.get_prices(tokens)) if not for_swaps else self.prepare_pools_for_swap(pools, tokens)
    
    
    @require_async_context
    async def get_pools_for_swaps(self) -> List[LiquidityPoolForSwap]: return await self.get_pools(for_swaps=True)

    @require_async_context
    async def _get_quotes_for_paths(self, from_token: Token, to_token: Token, amount_in: int, pools: List[LiquidityPoolForSwap], paths: List[List[Tuple]]) -> List[Optional[Quote]]:
        pools_dict = {p.lp: p for p in pools}
        path_pools = [list(map(lambda p: pools_dict[p[2]], path)) for path in paths]
        async with self.web3.batch_requests() as batch:
            batch = self.prepare_quote_batch(batch, path_pools, amount_in, paths)
            return self.prepare_quotes(from_token, to_token, amount_in, path_pools, await batch.async_execute())

    @require_async_context
    async def get_quote(self, from_token: Token, to_token: Token, amount_in: int, exclude_tokens: List[str] = []) -> Optional[Quote]:
        pools = await self.get_pools_for_swaps()
        paths = self.get_paths_for_quote(from_token, to_token, pools, exclude_tokens)
        quotes = sum(await asyncio.gather(*[self._get_quotes_for_paths(from_token, to_token, amount_in, pools, paths) for paths in chunk(paths, 500)]), [])
        return max(quotes, key=lambda q: q.amount_out) if len(quotes) > 0 else None

    @require_async_context
    async def set_token_allowance(self, token: Token, addr: str, amount: int):
        ERC20_ABI = [{
            "name": "approve",
            "type": "function",
            "constant": False,
            "inputs": [{"name": "spender", "type": "address"}, {"name": "amount", "type": "uint256"}],
            "outputs": [{"name": "", "type": "bool"}]
        }]
        token_contract = self.web3.eth.contract(address=token.token_address, abi=ERC20_ABI)
        return await self.sign_and_send_tx(token_contract.functions.approve(addr, amount))

    @require_async_context
    async def check_token_allowance(self, token: Token, addr: str) -> int:
        ERC20_ABI = [{
            "name": "allowance",
            "type": "function",
            "constant": True,
            "inputs": [{"name": "owner", "type": "address"}, {"name": "spender", "type": "address"}],
            "outputs": [{"name": "", "type": "uint256"}]
        }]
        token_contract = self.web3.eth.contract(address=token.token_address, abi=ERC20_ABI)
        return await token_contract.functions.allowance(self.account.address, addr).call()

    @require_async_context
    async def sign_and_send_tx(self, tx, value: int = 0, wait: bool = True):
        print(f"sign_and_send_tx: {tx} with value: {value}")
        spender = self.account.address
        tx = await tx.build_transaction({ 'from': spender, 'value': value, 'nonce': await self.web3.eth.get_transaction_count(spender) })
        signed_tx = self.account.sign_transaction(tx)
        tx_hash = await self.web3.eth.send_raw_transaction(signed_tx.raw_transaction)
        return await self.web3.eth.wait_for_transaction_receipt(tx_hash) if wait else tx_hash
    
    @require_async_context
    async def deposit(self, deposit: Deposit, delay_in_minutes: float = 30, slippage: float = 0.01):
        amount_token0, pool, router_contract_addr = deposit.amount_token0, deposit.pool, self.settings.router_contract_addr
        print(f"gonna deposit {amount_token0} {pool.token0.symbol} into {pool.symbol} from {self.account.address}")
        [token0_amount, token1_amount, _] = await self.router.functions.quoteAddLiquidity(
            pool.token0.token_address,
            pool.token1.token_address,
            pool.is_stable,
            pool.factory,
            float_to_uint256(amount_token0, pool.token0.decimals),
            MAX_UINT256
        ).call()
        print(f"Quote: {pool.token0.symbol} {token0_amount / 10 ** pool.token0.decimals} -> {pool.token1.symbol} {token1_amount / 10 ** pool.token1.decimals}")

        # set up allowance for both tokens
        print(f"setting up allowance for {pool.token0.symbol}")
        await self.set_token_allowance(pool.token0, router_contract_addr, token0_amount)

        print(f"setting up allowance for {pool.token1.symbol}")
        await self.set_token_allowance(pool.token1, router_contract_addr, token1_amount)

        # check allowances
        token0_allowance = await self.check_token_allowance(pool.token0, router_contract_addr)
        token1_allowance = await self.check_token_allowance(pool.token1, router_contract_addr)

        print(f"allowances: {token0_allowance}, {token1_allowance}")

        # adding liquidity

        # if token 0 is native, use addLiquidityETH instead of standard addLiquidity
        if pool.token0.token_address == self.settings.wrapped_native_token_addr:
            params = [
                pool.token1.token_address,
                pool.is_stable,
                token1_amount,
                apply_slippage(token1_amount, slippage),
                apply_slippage(token0_amount, slippage),
                self.account.address,
                get_future_timestamp(delay_in_minutes)
            ]
            print(f"adding liquidity with params: {params}")
            return await self.sign_and_send_tx(self.router.functions.addLiquidityETH(*params), value=token0_amount)
        
        # token 1 is native, use addLiquidityETH instead of standard addLiquidity
        if pool.token1.token_address == self.settings.wrapped_native_token_addr:
            params = [
                pool.token0.token_address,
                pool.is_stable,
                token0_amount,
                apply_slippage(token0_amount, slippage),
                apply_slippage(token1_amount, slippage),
                self.account.address,
                get_future_timestamp(delay_in_minutes)
            ]
            print(f"adding liquidity with params: {params}")
            return await self.sign_and_send_tx(self.router.functions.addLiquidityETH(*params), value=token1_amount)

        params = [
            pool.token0.token_address,
            pool.token1.token_address,
            pool.is_stable,
            token0_amount,
            token1_amount,
            apply_slippage(token0_amount, slippage),
            apply_slippage(token1_amount, slippage),
            self.account.address,
            get_future_timestamp(delay_in_minutes)
        ]

        print(f"adding liquidity with params: {params}")

        return await self.sign_and_send_tx(self.router.functions.addLiquidity(*params))

## Sync chain

In [ ]:
#| export

class Chain(CommonChain):
    web3: Web3
    sugar: Contract
    router: Contract
    slipstream: Contract
    prices: Contract
    quoter: Contract

    def __enter__(self):
        """Sync context manager entry"""
        self._in_context = True
        self.web3 = Web3(HTTPProvider(self.settings.rpc_uri))
        self.sugar = self.web3.eth.contract(address=self.settings.sugar_contract_addr, abi=sugar)
        self.slipstream = self.web3.eth.contract(address=self.settings.slipstream_contract_addr, abi=slipstream)
        self.prices = self.web3.eth.contract(address=self.settings.price_oracle_contract_addr, abi=price_oracle)
        self.router = self.web3.eth.contract(address=self.settings.router_contract_addr, abi=router)
        self.quoter = self.web3.eth.contract(address=self.settings.quoter_contract_addr, abi=quoter)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Sync context manager exit"""
        self._in_context = False
        return None
    
    @require_context
    def get_all_tokens(self, listed_only: bool = True) -> List[Token]:
        return self.prepare_tokens(self.sugar.functions.tokens(800, 0, ADDRESS_ZERO, []).call(), listed_only)
    
    # @cache_in_seconds(ORACLE_PRICES_CACHE_MINUTES * 60)
    def _get_prices(self, tokens: Tuple[Token]) -> List[int]:
        # token_address => normalized rate
        return self.prices.functions.getManyRatesToEthWithCustomConnectors(
            list(map(lambda t: t.wrapped_token_address or t.token_address, tokens)),
            False, # use wrappers
            self.settings.connector_tokens_addrs,
            10 # threshold_filter
        ).call()

    @require_context
    def get_prices(self, tokens: List[Token]) -> List[Price]:
        """Get prices for tokens in target stable token"""
        return self.prepare_prices(tokens, sum([self._get_prices(tuple(ts)) for ts in list(chunk(tokens, self.settings.price_batch_size))], []))
    
    @require_context
    def get_pools(self, for_swaps: bool = False) -> List[LiquidityPool]:
        pools, offset, limit = [], 0, self.settings.pool_page_size
        tokens = self.get_all_tokens(listed_only=False)

        while True:
            f = self.sugar.functions.all if not for_swaps else self.sugar.functions.forSwaps
            pools_batch = f(limit, offset).call()
            pools += pools_batch
            if len(pools_batch) == 0: break
            else: offset += limit

        return self.prepare_pools(pools, tokens, self.get_prices(tokens)) if not for_swaps else self.prepare_pools_for_swap(pools, tokens)
    
    @require_context
    def get_pools_for_swaps(self) -> List[LiquidityPoolForSwap]: return self.get_pools(for_swaps=True)

    @require_context
    def _get_quotes_for_paths(self, from_token: Token, to_token: Token, amount_in: int, pools: List[LiquidityPoolForSwap], paths: List[List[Tuple]]) -> List[Optional[Quote]]:
        pools_dict = {p.lp: p for p in pools}
        path_pools = [list(map(lambda p: pools_dict[p[2]], path)) for path in paths]
        with self.web3.batch_requests() as batch:
            batch = self.prepare_quote_batch(batch, path_pools, amount_in, paths)
            return self.prepare_quotes(from_token, to_token, amount_in, path_pools, batch.execute())

    @require_context
    def get_quote(self, from_token: Token, to_token: Token, amount_in: int, exclude_tokens: List[str] = []) -> Optional[Quote]:
        pools = self.get_pools_for_swaps()
        paths = self.get_paths_for_quote(from_token, to_token, pools, exclude_tokens)
        path_chunks = list(chunk(paths, 500))
        
        # Store self reference for thread context
        chain_instance = self
        
        def get_quotes_for_chunk(paths_chunk):
            """Thread-safe function to get quotes for a chunk of paths"""
            return chain_instance._get_quotes_for_paths(from_token, to_token, amount_in, pools, paths_chunk)
        
        # Collect all quotes from all threads
        all_quotes = []
        # with ThreadPoolExecutor() as executor:
        #     futures = [executor.submit(get_quotes_for_chunk, pc) for pc in path_chunks]
        #     for future in concurrent.futures.as_completed(futures):
        #         try:
        #             quotes_batch = future.result()
        #             all_quotes.extend(quotes_batch)
        #         except Exception as e:
        #             print(f"Error getting quotes: {e}")
        
        # # Return best quote
        # return max(all_quotes, key=lambda q: q.amount_out) if len(all_quotes) > 0 else None

        for pc in path_chunks:
            quotes = get_quotes_for_chunk(pc)
            for q in quotes: all_quotes.append(q)
            # print(f">>>>>>> {}")

        return max(all_quotes, key=lambda q: q.amount_out) if len(all_quotes) > 0 else None
        

## OP Chain

In [ ]:
#| export

class OPChainCommon():
    usdc: str = normalize_address("0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85")
    velo: str = normalize_address("0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db")

class AsyncOPChain(AsyncChain, OPChainCommon):
    def __init__(self, **kwargs): super().__init__(make_op_chain_settings(**kwargs))

class OPChain(Chain, OPChainCommon):
    def __init__(self, **kwargs): super().__init__(make_op_chain_settings(**kwargs))


In [ ]:
with OPChain() as op:
    tokens = op.get_all_tokens(listed_only=True)
    def get_token_by_addres(addr): return next(filter(lambda t: t.token_address == addr, tokens), None)
    from_token_addr, to_token_addr = normalize_address("0x9560e827af36c94d2ac33a39bce1fe78631088db"), normalize_address("0x0b2c639c533813f4aa9d7837caf62653d097ff85") 
    from_token, to_token, amount_in = get_token_by_addres(from_token_addr), get_token_by_addres(to_token_addr), float_to_uint256(100)
    exclude_tokens = ["0x74ccbe53f77b08632ce0cb91d3a545bf6b8e0979", "0x139283255069ea5deef6170699aaef7139526f1f", "0x88a89866439f4c2830986b79cbe6f69d1bd548bb", "0x8901cb2e82cc95c01e42206f8d1f417fe53e7af0"]
    q = op.get_quote(from_token, to_token, amount_in, exclude_tokens)
    if q:
        print(f"{q.amount_in} -> {q.amount_out / 10 ** to_token.decimals}")
        print(' - '.join(map(lambda p: p.symbol, q.path)))

100000000000000000000 -> 3.783388
CL50-VELO/opxVELO - CL200-USDC/opxVELO


## Base Chain

In [ ]:
#| export

class BaseChainCommon():
    usdc: str = normalize_address("0x833589fcd6edb6e08f4c7c32d4f71b54bda02913")
    aero: str = normalize_address("0x940181a94a35a4569e4529a3cdfb74e38fd98631")

class AsyncBaseChain(AsyncChain, BaseChainCommon):
    def __init__(self, **kwargs): super().__init__(make_base_chain_settings(**kwargs))

class BaseChain(Chain, BaseChainCommon):
    def __init__(self, **kwargs): super().__init__(make_base_chain_settings(**kwargs))

## Simnet versions of chains

Simnet URIs:

- OP: http://127.0.0.1:4444
- Base: http://127.0.0.1:4445

This assumes the following setup:

```
supersim fork  --l2.host=0.0.0.0 --l2.starting.port=4444 --chains=op,base
```

In [ ]:
#| export 

class AsyncOPChainSimnet(AsyncOPChain):
    def __init__(self,  **kwargs): super().__init__(rpc_uri="http://127.0.0.1:4444", **kwargs)

class AsyncBaseChainSimnet(AsyncBaseChain):
    def __init__(self,  **kwargs): super().__init__(rpc_uri="http://127.0.0.1:4445", **kwargs)

## Tests

Run tests using mainnets for reads and simnet for writes

Make sure simnet is running

In [ ]:
from fastcore.test import test_eq
import socket

host, port ='127.0.0.1', 4444
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex((host, port))
    test_eq(result, 0)
except socket.error as err:
    test_eq(err, None)
finally:
    sock.close()

AssertionError: ==:
61
0

Make sure PK is set for writes

In [ ]:
import os
from fastcore.test import test_ne
from dotenv import load_dotenv

load_dotenv()

test_ne(os.getenv("SUGAR_PK"), None)

Getting tokens. Make sure native token is included

In [ ]:
from fastcore.test import test_eq, test_ne

async with AsyncOPChain() as op:
    tokens = await op.get_all_tokens()
    native_token = next(filter(lambda t: t.is_native, tokens), None)
    test_ne(native_token, None)
    test_eq(native_token.symbol, "ETH")

with OPChain() as op_sync:
    tokens = op_sync.get_all_tokens()
    native_token = next(filter(lambda t: t.is_native, tokens), None)
    test_ne(native_token, None)
    test_eq(native_token.symbol, "ETH")

async with AsyncBaseChain() as base:
    tokens = await base.get_all_tokens()
    native_token = next(filter(lambda t: t.is_native, tokens), None)
    test_ne(native_token, None)
    test_eq(native_token.symbol, "ETH")

Check on tokens and pools

In [ ]:
async with AsyncOPChain() as chain:
    tokens = await chain.get_all_tokens()
    prices = await chain.get_prices(tokens)
    pools = await chain.get_pools()

    for p in prices[:5]: print(f"{p.token.symbol} price: ${p.price}")
    for p in pools[:5]: print(f"{p.symbol} pool: ${p.tvl}")

Quotes

In [ ]:
async with AsyncOPChain() as op:
    tokens = await op.get_all_tokens(listed_only=True)
    def get_token_by_addres(addr): return next(filter(lambda t: t.token_address == addr, tokens), None)
    from_token_addr, to_token_addr = normalize_address("0x9560e827af36c94d2ac33a39bce1fe78631088db"), normalize_address("0x0b2c639c533813f4aa9d7837caf62653d097ff85") 
    from_token, to_token, amount_in = get_token_by_addres(from_token_addr), get_token_by_addres(to_token_addr), float_to_uint256(100)
    exclude_tokens = ["0x74ccbe53f77b08632ce0cb91d3a545bf6b8e0979", "0x139283255069ea5deef6170699aaef7139526f1f", "0x88a89866439f4c2830986b79cbe6f69d1bd548bb", "0x8901cb2e82cc95c01e42206f8d1f417fe53e7af0"]
    q = await op.get_quote(from_token, to_token, amount_in, exclude_tokens)
    if q:
        print(f"{q.amount_in} -> {q.amount_out / 10 ** to_token.decimals}")
        print(' - '.join(map(lambda p: p.symbol, q.path)))

100000000000000000000 -> 3.751304
vAMM-USDC/VELO


In [ ]:
with OPChain() as chain:
    tokens = chain.get_all_tokens()
    prices = chain.get_prices(tokens)
    pools = chain.get_pools()

    for p in prices[:5]: print(f"{p.token.symbol} price: ${p.price}")
    for p in pools[:5]: print(f"{p.symbol} pool: ${p.tvl}")

Basic deposit on OP

In [ ]:
async with AsyncOPChain() as chain:
    pools = await chain.get_pools()
    pools = list(filter(lambda x: x.token0 and x.token0.token_address == chain.usdc and x.token1.token_address == chain.velo, pools))
    async with AsyncOPChainSimnet() as supersim:
        print(f"supersim is {supersim}")
        # 0.02 USDC 
        await supersim.deposit(Deposit(pools[0], 0.02))

Native -> non Native on OP

In [ ]:
## WIP
# async with OPChain() as op:
#     pools = await op.get_pools()
#     # find WETH/Velo pool
#     weth_velo_pool = next(filter(lambda p: p.token0.token_address == op.settings.wrapped_native_token_addr and p.token1.token_address == OPChain.velo, pools), None)
#     test_ne(weth_velo_pool, None)
#     async with OPChainSimnet() as supersim:
#         await supersim.deposit(Deposit(weth_velo_pool, 0.0001))


Basic deposit on Base

In [ ]:
async with AsyncBaseChain() as chain:
    pools = await chain.get_pools()
    pools = list(filter(lambda x: x.token0 and x.token0.token_address == chain.usdc and x.token1.token_address == chain.aero, pools))
    async with AsyncBaseChainSimnet() as supersim:
        # 0.02 USDC 
        await supersim.deposit(Deposit(pools[0], 0.01), slippage=0.05)

CL on OP

In [ ]:
# WIP
# async with OPChain() as chain:
#     pools = await chain.get_pools()
#     # CL200-USDC/VELO
#     cl_usdc_velo = next(filter(lambda x: x.token0 and x.token0.token_address == chain.usdc and x.token1.token_address == chain.velo and x.is_cl, pools), None)
#     test_ne(cl_usdc_velo, None)
#     async with OPChainSimnet() as supersim:
#         await supersim.deposit(Deposit(cl_usdc_velo, 0.02))


In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()